In [56]:
import pandas as pd
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import figure

# read yritysdata
df = pd.read_csv('./ydata4.csv', header=0, sep=';')

# shape and data types of the data
print(df.shape)
#print(df.dtypes)

(16588, 67)


C:\Users\Tuomas\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (6,7,27,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# vanha osoitedata
# download csv
#url = 'https://kartta.hel.fi/ws/geoserver/avoindata/wfs?version=1.1.0&request=GetFeature&typeName=avoindata:Helsinki_nimisto&outputformat=csv'
#urllib.request.urlretrieve(url, 'wfsdata.csv')

In [10]:
# read osoitedata (Postin avointa dataa)
df_paikat = pd.read_csv('./BAF_new.csv', header=0, sep=';')

In [11]:
print(df_paikat.shape)
print(df_paikat.dtypes)

(405249, 5)
postinro        int64
kaupunki_fi    object
kaupunki_sv    object
osoite_fi      object
osoite_sv      object
dtype: object


In [13]:
# turhien rivien poisto osoitedatasta, esim. postilokerot PL/PB

df_paikat['osoite_fi'].replace('', np.nan, inplace=True)
df_paikat = df_paikat[~df_paikat.osoite_fi.str.contains("PL\s\d", case=True, na=False, regex=True)]
df_paikat = df_paikat[~df_paikat.osoite_fi.str.contains("PL\d", case=True, na=False, regex=True)]
df_paikat = df_paikat[~df_paikat.osoite_fi.str.contains("PB\s\d", case=True, na=False, regex=True)]
df_paikat = df_paikat[~df_paikat.osoite_fi.str.contains("PB\d", case=True, na=False, regex=True)]

In [14]:
# esimerkki Postin avoimesta osoitedatasta
df_paikat.reset_index(drop=True, inplace=True)
df_paikat.iloc[100:115]

,postinro,kaupunki_fi,kaupunki_sv,osoite_fi,osoite_sv
100,100,HELSINKI,HELSINGFORS,Annankatu,Annegatan
101,100,HELSINKI,HELSINGFORS,Itäinen Teatterikuja,Östra Teatergränden
102,100,HELSINKI,HELSINGFORS,Itäinen Teatterikuja,Östra Teatergränden
103,100,HELSINKI,HELSINGFORS,Kansalaistori,Medborgartorget
104,100,HELSINKI,HELSINGFORS,Kluuvikatu,Glogatan
105,100,HELSINKI,HELSINGFORS,Kluuvikatu,Glogatan
106,100,HELSINKI,HELSINGFORS,Mikonkatu,Mikaelsgatan
107,100,HELSINKI,HELSINGFORS,Mikonkatu,Mikaelsgatan
108,100,HELSINKI,HELSINGFORS,Eteläinen Rautatiekatu,Södra Järnvägsgatan
109,100,HELSINKI,HELSINGFORS,J. R. Aspelinin aukio,J. R. Aspelins plats


In [15]:
# rajaa kohdedatasta vain postiosoite, postinumero, postitoimipaikka
df2 = df[['K_KATUNIMI', 'K_POSTINUMERO', 'K_POSTITOIMIPAIKKA']].copy()

In [16]:
#df2.head(5)

In [17]:
df2.dtypes

K_KATUNIMI             object
K_POSTINUMERO         float64
K_POSTITOIMIPAIKKA     object
dtype: object

In [18]:
df2.shape

(16588, 3)

In [19]:
# missing data suhteet

for col in df2.columns:
    pct_missing = np.mean(df2[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

K_KATUNIMI - 19.0%
K_POSTINUMERO - 19.0%
K_POSTITOIMIPAIKKA - 19.0%


In [20]:
for col in df_paikat.columns:
    pct_missing = np.mean(df_paikat[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

postinro - 0.0%
kaupunki_fi - 0.0%
kaupunki_sv - 0.0%
osoite_fi - 2.0%
osoite_sv - 85.0%


In [21]:
# dropna df2, jos postiosoite tyhjä
df2.dropna(subset=['K_KATUNIMI'], inplace=True)

In [22]:
# postinro dtype vaihto integer
df2['K_POSTINUMERO'] = df2['K_POSTINUMERO'].fillna(0).astype(np.int64)

In [23]:
#df2.head(5)

In [24]:
# poistetaan turhat rivit (esim. PL tai Poste Restante katunimi-kentässä)
# tärkeämpi postiosoitteiden kanssa

df2 = df2[~df2.K_KATUNIMI.str.contains("PL\s\d", case=True, na=False, regex=True)]
df2 = df2[~df2.K_KATUNIMI.str.contains("PL\d", case=True, na=False, regex=True)]
df2 = df2[~df2.K_KATUNIMI.str.contains("PB\s\d", case=True, na=False, regex=True)]
df2 = df2[~df2.K_KATUNIMI.str.contains("PB\d", case=True, na=False, regex=True)]
df2 = df2[~df2.K_KATUNIMI.str.contains('Poste Restante', na=False)]
df2 = df2[~df2.K_KATUNIMI.str.contains("\-", case=True, na=False, regex=True)]

In [25]:
# listataan kaikki viralliset kaupungit suomeksi ja ruotsiksi
kaupungit = np.concatenate([df_paikat['kaupunki_fi'].unique() , df_paikat['kaupunki_sv'].unique()], axis=0)
kaupungit = pd.Series(kaupungit)

# otetaan mukaan vain rivit, joissa postitoimipk löytyy listalta
df2 = df2[df2.K_POSTITOIMIPAIKKA.isin(kaupungit)]

In [26]:
# Vain postiosoitteille:
# splitataan postiosoite-sarake (katu erilleen)
  
# temporary dataframe
# ensimmäisen numeron kohdalta poikki
#new = df2['POSTIOSOITE'].str.split("\s(?=\d)", n=1, expand=True)

# new columns
#df2['POSTIOSOITE_KATU'] = new[0]  
#df2['POSTIOSOITE_NRO'] = new[1] 

# dropataan vanha postiosoite
#df2.drop(columns =["POSTIOSOITE"], inplace = True) 

In [27]:
# Vain postiosoitteille:
# otetaan talteen rivit, joissa postiosoitteella ei kadunnumeroa (ei validi osoite)
# poisteteaan nämä df2

#df_nonumber = df2.loc[df_paikat['POSTIOSOITE_NRO'] == ""].copy()
#df2.dropna(subset=['POSTIOSOITE_NRO'], inplace=True)

In [28]:
# lisätään name_not_found column
df2['name_not_found'] = np.nan

df2.reset_index(drop=True, inplace=True)

In [31]:
# luo flag jokaiseen osoitteeseen
# löytyy osoitekannasta (true) / ei löydy (false)

def checkIfFound(city, street):
    current = df_paikat.loc[df_paikat['kaupunki_fi'] == city].append(df_paikat.loc[df_paikat['kaupunki_sv'] == city])
    streets_current = current['osoite_fi'].dropna().append(current['osoite_sv'].dropna())
    if street in streets_current[streets_current.str.startswith(street[0])].unique():
        return 'False'
    else:
        return 'True'
    
df2['name_not_found'] = df2.apply(lambda row: checkIfFound(row['K_POSTITOIMIPAIKKA'], row['K_KATUNIMI']), axis=1)

In [32]:
#df2.head(50)

In [33]:
# käsitellään vain virheellisiä

df2_notfound = df2[df2['name_not_found'] == 'True']
#for i in df2_notfound.values:
#    print(i)

In [34]:
# löydettyjen virheellisten lkm
len(df2_notfound)

358

In [35]:
#df2_notfound.head(50)

In [36]:
# nltk imports
from nltk.corpus import words
from nltk.metrics.distance import (
    edit_distance,
    jaccard_distance,
    )
from nltk.util import ngrams

df2_notfound['jaccard'] = np.nan
df2_notfound['jaccard_ng'] = np.nan
df2_notfound['edit_distance'] = np.nan
df2_notfound.reset_index(drop=True, inplace=True)

C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [37]:
# autocorrectin kaltainen systeemi
# funktiot käyttävät sanakirjan sijaan listaa oikeista osoitteista
# vertaillaan vuorotellen jokaista käsiteltävää osoitetta virallisen kannan osoitteisiin

def jaccard_ng(toimipk, entry, gram_number):
    df_paikat_current = df_paikat.loc[(df_paikat['kaupunki_fi'] == toimipk) | (df_paikat['kaupunki_sv'] == toimipk)]
    spellings = df_paikat_current['osoite_fi'].dropna().append(df_paikat_current['osoite_sv'].dropna())
    spellings = spellings[spellings.str.startswith(entry[0])] # vertaillaan samalla alkukirjaimella
    
    distances = ((jaccard_distance(set(ngrams(entry, gram_number)), 
                                   set(ngrams(word, gram_number))), word) for word in spellings)
    closest = min(distances, default="EMPTY") # empty poikkeusten varalle
    if closest != 'EMPTY':
        return closest[1]
        
def jaccard(toimipk, entry):
    df_paikat_current = df_paikat.loc[(df_paikat['kaupunki_fi'] == toimipk) | (df_paikat['kaupunki_sv'] == toimipk)]
    spellings = df_paikat_current['osoite_fi'].dropna().append(df_paikat_current['osoite_sv'].dropna())
    spellings = spellings[spellings.str.startswith(entry[0])] # vertaillaan samalla alkukirjaimella
    
    distances = ((jaccard_distance(set(entry), set(word)), word) for word in spellings)
    closest = min(distances, default="EMPTY")
    #print(closest)
    if closest != 'EMPTY':
        return closest[1]        

def edit_dist(toimipk, entry):
    df_paikat_current = df_paikat.loc[(df_paikat['kaupunki_fi'] == toimipk) | (df_paikat['kaupunki_sv'] == toimipk)]
    spellings = df_paikat_current['osoite_fi'].dropna().append(df_paikat_current['osoite_sv'].dropna())
    spellings = spellings[spellings.str.startswith(entry[0])] # vertaillaan samalla alkukirjaimella
    
    distances = ((edit_distance(entry, word), word) for word in spellings)
    closest = min(distances, default="EMPTY")
    #print(closest)
    if closest != 'EMPTY':
        return closest[1]

In [38]:
df2_notfound['jaccard'] = df2_notfound.apply(lambda row: jaccard(row['K_POSTITOIMIPAIKKA'], row['K_KATUNIMI']), axis=1)

C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df2_notfound['jaccard_ng'] = df2_notfound.apply(lambda row: jaccard_ng(row['K_POSTITOIMIPAIKKA'], row['K_KATUNIMI'], 2), axis=1)

C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
df2_notfound['edit_distance'] = df2_notfound.apply(lambda row: edit_dist(row['K_POSTITOIMIPAIKKA'], row['K_KATUNIMI']), axis=1)

C:\Users\Tuomas\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
# muutama esimerkki tuloksista

df2_notfound.iloc[150:162]
#df2_notfound.head(50)

,K_KATUNIMI,K_POSTINUMERO,K_POSTITOIMIPAIKKA,name_not_found,jaccard,jaccard_ng,edit_distance
150,Porpuripolku,420,HELSINKI,True,Purpuripolku,Purpuripolku,Purpuripolku
151,Franzeninkatu,500,HELSINKI,True,Franzéninkatu,Franzéninkatu,Franzéninkatu
152,Neljäs Linja,530,HELSINKI,True,Neljäs linja,Neljäs linja,Neljäs linja
153,Stålbergintie,570,HELSINKI,True,Ståhlbergintie,Ståhlbergintie,Ståhlbergintie
154,Vanha Talvitie,580,HELSINKI,True,Vanha talvitie,Vanha talvitie,Vanha talvitie
155,Laivalahden Puistotie,810,HELSINKI,True,Laivalahden puistotie,Laivalahden puistotie,Laivalahden puistotie
156,Kasteholmantie,900,HELSINKI,True,Kastelholmantie,Kastelholmantie,Kastelholmantie
157,Liikkalalntie,950,HELSINKI,True,Liikkalantie,Liikkalantie,Liikkalantie
158,Hallipiha,1450,VANTAA,True,Hallinpiha,Hallinpiha,Hallinpiha
159,Raapavuorenpolku,1620,VANTAA,True,Raappavuorenpolku,Raappavuorenpolku,Raappavuorenpolku


In [42]:
# test
jc = (jaccard_distance(set('Lukkarinmäen tie'), set('Lukkarimäenkuja')))
print(jc)

0.23076923076923078


In [43]:
# into csv
df2_notfound.to_csv(r'./testi_dataset2.csv', index = False, header=True)